In [61]:
import pandas as pd

In [62]:
df = pd.read_csv('../data/2_taxi_nyc.csv', encoding='utf-8', sep=',')

In [63]:
df.head(3)

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [64]:
df.shape

(29101, 14)

In [69]:
df.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [71]:
df = df.rename(columns={
    'pcp 01': 'pcp_01',
    'pcp 06': 'pcp_06',
    'pcp 24': 'pcp_24'
})

In [72]:
df.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

In [86]:
counter_notes_by_borough = df.value_counts('borough')
counter_notes_by_borough

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

In [84]:
counter_notes_by_borough['Manhattan']

4343

In [85]:
counter_notes_by_borough['Brooklyn']

4343

In [88]:
all_pickups = df['pickups'].sum()
all_pickups

14265773

In [106]:
borough_by_pickups = df.groupby('borough', as_index=False) \
    .aggregate({'pickups': 'sum'}) \
    .sort_values('pickups', ascending=False)

In [137]:
borough_by_pickups

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


In [128]:
name_of_borough_by_min_pickups = borough_by_pickups.loc[
    borough_by_pickups['pickups'].idxmin()
]['borough']

name_of_borough_by_min_pickups

'EWR'

In [158]:
borough_by_pickups_and_hday = df.groupby(['borough', 'hday'], as_index=False) \
    .aggregate({'pickups': 'mean'}) \
    .sort_values('pickups', ascending=False)

borough_by_pickups_and_hday

,borough,hday,pickups
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
9,Queens,Y,320.730539
8,Queens,N,308.899904
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
10,Staten Island,N,1.606082
11,Staten Island,Y,1.497006


In [167]:
weekdays = borough_by_pickups_and_hday \
    .query("hday == 'N'")[['borough', 'pickups']] \
        .rename(columns={'pickups': 'weekdays_avg'})
weekends = borough_by_pickups_and_hday \
    .query("hday == 'Y'")[['borough', 'pickups']] \
        .rename(columns={'pickups': 'weekends_avg'})

In [168]:
merged_df = weekends.merge(weekdays, on='borough')

In [172]:
merged_df.query('weekends_avg > weekdays_avg')

,borough,weekends_avg,weekdays_avg
2,Queens,320.730539,308.899904
5,EWR,0.041916,0.023467


In [177]:
pickups_by_month = df.groupby(['pickup_month', 'borough']) \
    .aggregate({'pickups': 'sum'}) \
    .sort_values('pickups', ascending=False)

In [179]:
def temperature_to_celcius(fahrengeit):
    return (fahrengeit - 32) / 1.8


df['temp_C'] = temperature_to_celcius(df['temp'])
df['temp_C'][:5]

0   -1.111111
1   -1.111111
2   -1.111111
3   -1.111111
4   -1.111111
Name: temp_C, dtype: float64